In [ ]:
!pip install transformers datasets pandas scikit-learn accelerate peft bitsandbytes evaluate


In [ ]:
import pandas as pd
from datasets import Dataset, ClassLabel
from sklearn.model_selection import train_test_split

# Load the data from your CSV file. Make sure 'dataset_odia.csv' is in the same directory.
df = pd.read_csv('dataset_odia.csv')

# Map the string labels ("wait", "unfinished", "finished") to integer IDs.
label_map = {"wait": 0, "unfinished": 1, "finished": 2}
df['label'] = df['label'].map(label_map)

# Create a Hugging Face Dataset object.
df = df.rename(columns={'sentence': 'text'})
dataset = Dataset.from_pandas(df)

# Define the class labels in the dataset.
dataset = dataset.cast_column(
    "label", ClassLabel(names=["wait", "unfinished", "finished"])
)


Casting the dataset:   0%|          | 0/9999 [00:00<?, ? examples/s]

In [ ]:
train_temp_split = dataset.train_test_split(test_size=0.2, seed=42)
valid_test_split = train_temp_split["test"].train_test_split(test_size=0.5, seed=42)

train_dataset = train_temp_split["train"]
valid_dataset = valid_test_split["train"]
test_dataset  = valid_test_split["test"]

print("Training dataset:", train_dataset)
print("Validation dataset:", valid_dataset)
print("Test dataset:", test_dataset)

Training dataset: Dataset({
    features: ['text', 'label'],
    num_rows: 7999
})
Validation dataset: Dataset({
    features: ['text', 'label'],
    num_rows: 1000
})
Test dataset: Dataset({
    features: ['text', 'label'],
    num_rows: 1000
})


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

model_name = "livekit/turn-detector"
num_labels = 3

# Configure 4-bit quantization to reduce memory usage.
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# Load the pre-trained model with quantization settings.
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    quantization_config=bnb_config,
    device_map="auto",
)

# Configure LoRA with the correct target modules for a Llama-based architecture.
peft_config = LoraConfig(
    r=8,  # LoRA rank
    lora_alpha=16, # Scaling factor
    target_modules=["q_proj", "k_proj", "v_proj"], # The attention projection layers
    lora_dropout=0.05,
    bias="none",
)

# Prepare the model for k-bit training and wrap it with the LoRA adapters.
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

# Load the tokenizer.
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at livekit/turn-detector and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!pip install evaluate

In [ ]:
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate

# Define a function to tokenize the text.
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length = 128)

# Apply the tokenizer to datasets.
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_valid = valid_dataset.map(tokenize_function, batched=True)
tokenized_test  = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/7999 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
!pip install -U transformers

In [ ]:
def compute_metrics(eval_pred):
    metric = evaluate.load("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Set training arguments for the Trainer.
training_args = TrainingArguments(
    output_dir="./odia_turn_detector_qlora",
   eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none",
    gradient_accumulation_steps=1,
    fp16=True,
    dataloader_num_workers=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

/tmp/ipython-input-3237503533.py:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Accuracy
1,0.800100,0.785045,0.757000
2,0.395800,0.364471,0.938000
3,0.234500,0.228927,0.973000


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=1500, training_loss=0.6107111473083496, metrics={'train_runtime': 651.2676, 'train_samples_per_second': 36.847, 'train_steps_per_second': 2.303, 'total_flos': 1969218620227584.0, 'train_loss': 0.6107111473083496, 'epoch': 3.0})

In [ ]:
test_results = trainer.evaluate(eval_dataset=tokenized_test)

print("Final Test Results:")
print(f"Loss: {test_results['eval_loss']:.4f}")
print(f"Accuracy: {test_results['eval_accuracy']:.4f}")

Final Test Results:
Loss: 0.2347
Accuracy: 0.9630


In [ ]:
output_dir = "./odia_turn_detector_qlora"
trainer.save_model(output_dir)

tokenizer.save_pretrained(output_dir)

('./odia_turn_detector_qlora/tokenizer_config.json',
 './odia_turn_detector_qlora/special_tokens_map.json',
 './odia_turn_detector_qlora/chat_template.jinja',
 './odia_turn_detector_qlora/vocab.json',
 './odia_turn_detector_qlora/merges.txt',
 './odia_turn_detector_qlora/added_tokens.json',
 './odia_turn_detector_qlora/tokenizer.json')

In [ ]:
import torch.nn.functional as F

# Paths
model_path = "/content/odia_turn_detector_qlora"
tokenizer = AutoTokenizer.from_pretrained(model_path)

model = AutoModelForSequenceClassification.from_pretrained(
    model_path,
    num_labels=3
)
model.eval()

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at livekit/turn-detector and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


LlamaForSequenceClassification(
  (model): LlamaModel(
    (embed_tokens): Embedding(49154, 576)
    (layers): ModuleList(
      (0-29): 30 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=576, out_features=576, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=576, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=576, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): lora.Linear(
            (base_layer): Linear(in_features=576, out_features=192, bias=False)
            (lora_dropout): ModuleDict(
              (def

In [ ]:
sentence = "ଆପଣଙ୍କ ପାଇଁ ପକାଉଛି ଭୋଜନ ମୁଁ?"

# Tokenize
inputs = tokenizer(
    sentence,
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=128
)
inputs = {k: v.to(device) for k, v in inputs.items()}

In [ ]:
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    probs = F.softmax(logits, dim=-1)
    pred_class_id = torch.argmax(probs, dim=-1).item()

In [ ]:
id2label = {0: "wait", 1: "unfinished", 2: "finished"}
pred_label = id2label[pred_class_id]

print(f"Sentence: {sentence}")
print(f"Predicted class: {pred_label}")
print(f"Probabilities: {probs.cpu().numpy()}")

Sentence: ଆପଣଙ୍କ ପାଇଁ ପକାଉଛି ଭୋଜନ ମୁଁ?
Predicted class: wait
Probabilities: [[0.87919825 0.08743788 0.03336393]]
